In [1]:
import numpy as np
import tensorflow as tf
from random import randrange
from os import startfile
from environment_generation import get_environment_state
from preprocessing import one_hot_encode_environment_state
from visualization import visualize_environment
from simulation import simulation, show_environment_state
from lawn_mowing_environment import environment

C:\Users\Jake\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Jake\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Jake\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Jake\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (

In [2]:
starting_environment_state = get_environment_state()
show_environment_state(starting_environment_state)

In [3]:
one_hot_encode_environment_state(starting_environment_state)

array([[0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.]])

In [ ]:
# initalize the simulation
agent_starting_position = [8,8]
agent_starting_direction = 'S'
sim = simulation(starting_environment_state, environment(agent_starting_position, agent_starting_direction, starting_environment_state.shape), visualize_environment())

In [ ]:
observation = sim.reset()

done = False
while not done:
    observation, reward, done = sim.step(randrange(1,4)) # take random action
print(reward)
sim.get_simulation_history_visualization("agent_videos/basic_mlp_agent.mp4", 60)

In [ ]:
startfile("agent_videos\\basic_mlp_agent.mp4")

In [ ]:
n_neurons_1 = 1024
n_neurons_2 = 512
n_neurons_3 = 256
n_neurons_4 = 128
n_features = 1
#begin the session & init placeholders
net = tf.Session()
X = tf.placeholder(dtype=tf.float32, shape=[None, n_features], name="X")
Y = tf.placeholder(dtype=tf.float32, shape=[None], name="Y")
# Initializers
weight_initializer = tf.variance_scaling_initializer(mode="fan_avg", distribution="uniform", scale=1)
bias_initializer = tf.zeros_initializer()
# Hidden weights
W_hidden_1 = tf.Variable(weight_initializer([n_features, n_neurons_1]))
bias_hidden_1 = tf.Variable(bias_initializer([n_neurons_1]))
W_hidden_2 = tf.Variable(weight_initializer([n_neurons_1, n_neurons_2]))
bias_hidden_2 = tf.Variable(bias_initializer([n_neurons_2]))
W_hidden_3 = tf.Variable(weight_initializer([n_neurons_2, n_neurons_3]))
bias_hidden_3 = tf.Variable(bias_initializer([n_neurons_3]))
W_hidden_4 = tf.Variable(weight_initializer([n_neurons_3, n_neurons_4]))
bias_hidden_4 = tf.Variable(bias_initializer([n_neurons_4]))
# Output weights
W_out = tf.Variable(weight_initializer([n_neurons_4, 1]))
bias_out = tf.Variable(bias_initializer([1]))
# Hidden layers
hidden_1 = tf.nn.relu(tf.add(tf.matmul(X, W_hidden_1), bias_hidden_1))
hidden_2 = tf.nn.relu(tf.add(tf.matmul(hidden_1, W_hidden_2), bias_hidden_2))
hidden_3 = tf.nn.relu(tf.add(tf.matmul(hidden_2, W_hidden_3), bias_hidden_3))
hidden_4 = tf.nn.relu(tf.add(tf.matmul(hidden_3, W_hidden_4), bias_hidden_4))
# Output layer
out = tf.transpose(tf.add(tf.matmul(hidden_4, W_out), bias_out), name="out")
# Loss function
mse = tf.reduce_mean(tf.squared_difference(out, Y, name="mse"))
# Optimizer
opt = tf.train.AdamOptimizer().minimize(mse)
# Init
net.run(tf.global_variables_initializer())
# Train
for e in tqdm(range(epochs)):
    net.run(opt, feed_dict={X: inputs, Y: outputs})
tf.train.Saver().save(net, model_location + model_name)

init_g = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()
# start tensorflow session
net = tf.Session()
net.run(init_g)
net.run(init_l)
# load model
saver = tf.train.import_meta_graph(model_location + model_name + '.meta')
saver.restore(net, model_location + model_name)
# re-initialize tensors
graph = tf.get_default_graph()
Y = graph.get_tensor_by_name("Y:0")
X = graph.get_tensor_by_name("X:0")
mse = graph.get_tensor_by_name("mse:0")
out = graph.get_tensor_by_name("out:0")
self.mse = np.array(net.run(mse, feed_dict={X: inputs, Y: outputs})).mean()
self.ppf = lambda x: (net.run(out, feed_dict={X: np.array([x]).T})[0])*100